In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### This notebook is based on "aligned interpolation" example from "diferrentiable image parametrizations" article on distill.pub

### The purpose is to research possibilities of aligned interpolation of feature vizualization by feeding the result of previous optimization to the next

## Install, Import, Load Model

In [ ]:
!pip install -q lucid>=0.2.3

In [ ]:
# Imports

import numpy as np
import tensorflow as tf

import lucid.modelzoo.vision_models as models
from lucid.misc.io import load, save, show

import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render

In [ ]:
from lucid.optvis.param import image, image_sample
from lucid.misc.tfutil import create_session

In [ ]:
# Let's import a model from the Lucid modelzoo!

model = models.InceptionV1()
model.load_graphdef()

## Creating initial image


In [ ]:
h, w = 256, 256

In [ ]:
 with tf.Graph().as_default() as graph, tf.Session() as sess:
  N1 = render.make_vis_T(model, 'mixed4d:1', lambda: param.image(h, w, sd=0.42))
  tf.initialize_all_variables().run()
  
  for i in range(512):
    N1("vis_op").run()
    if not i % 100:
      show(N1("input").eval())
  init_im = N1("input").eval()

In [ ]:
def render_from_image(objective, image, n_iter=100):
    """returns list of frames"""
    frames = []
  
    with tf.Graph().as_default() as graph, tf.Session() as sess:
      var = tf.Variable(image)

      T = render.make_vis_T(model, objective, lambda: var.value(), optimizer=tf.train.AdamOptimizer(learning_rate=0.01))
      tf.initialize_all_variables().run()

      for i in range(n_iter):
        T("vis_op").run()
        frames.append(T("input").eval())
      
      return frames

## Creating interpolated images


In [ ]:
vl = [init_im]
keyframes = [init_im]
obj = ('mixed4d', 2)
for i in range(10):
  new_obj = 'mixed4d', i + 3
  n = 16
  for j in range(n):
    inter_obj = j / n * objectives.channel(*new_obj) + n / (j+1) * objectives.channel(*obj)
    vl += render_from_image(inter_obj, vl[-1], n_iter=int(512 / n))
    keyframes.append(vl[-1])
    obj = new_obj
  show(vl[-1])


## Making Video

In [ ]:
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter

from IPython.display import clear_output, Image, display

In [ ]:
out_name = "video"
writer = FFMPEG_VideoWriter(out_name+'.mp4', (w, h), 60.0)
with writer:
  for vis in keyframes:
    writer.write_frame(np.uint8(vis*255.0))